In [1]:
#!pip install pypdf
# PDF file, using ChatGLM (GPU) + trans-opus-mt-en-zh (CPU)
# trans-opus-mt-en-zh is use for translate short content

In [2]:
from langchain.document_loaders import PyPDFLoader

In [3]:
loader = PyPDFLoader("Your_file_here.pdf")
pages = loader.load_and_split()

In [ ]:
#len(pages[0].dict()['page_content'])
l={}
c=0
print(f"{len(pages)}")
for x in pages:
    l[c]=len(x.dict()['page_content'])
    c += 1

In [ ]:
l2=sorted(l.values())
l2

In [ ]:
max(l, key=l.get)

In [ ]:
pages[max(l, key=l.get)].dict()['page_content']

In [ ]:
pages[17].dict()['page_content']

In [4]:
from langchain.prompts import PromptTemplate
template = """
翻译成中文:\n
{question}
"""
prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
# Load translate module
from transformers import AutoModelWithLMHead,AutoTokenizer,pipeline
mode_name = 'liam168/trans-opus-mt-en-zh'
model = AutoModelWithLMHead.from_pretrained(mode_name)
tokenizer = AutoTokenizer.from_pretrained(mode_name)
translation = pipeline("translation_en_to_zh", model=model, tokenizer=tokenizer, device="cpu")

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True).half().cuda()
model = model.eval()

In [7]:
from ipywidgets import IntProgress
from IPython.display import display

In [ ]:
p2 = IntProgress(max=len(pages))
p2.description = 'Running'
display(p2)
b=''
x=-1
# translated to 102
with open(r'Your_result_here.txt', 'a', encoding='UTF-8') as fp:
    for i in pages:
        x+=1
        p2.value+=1
        # Use debug mode to monitor "x" if you what to continue translation next time.
        # Replace '0' to pervious last status value. 
        if x > 0:
            max_len=len(i.dict()['page_content'])
            res = i.dict()['page_content']
            # Translate if text len is longer then 100
            if max_len > 100:
                # Use CPU for text len smaller then 900
                if max_len < 900:
                    temp=translation(i.dict()['page_content'], max_length=max_len)
                    res= temp[0]['translation_text']
                else:
                    res, history = model.chat(tokenizer, prompt.format(question=i.dict()['page_content']), history=[])
            b= b + res + '\n'
            fp.write('\n'+ res)
            fp.flush()
    fp.close()